### Working on GSOC

## Task 1: Social Media Data Extraction & Preprocessing (API Handling & Text Cleaning)


In [ ]:
#Importing the required libraries

import pandas as pd
import tweepy
import time

In [ ]:
### My keys

API_KEY = "MQl6JqRK2YTb1AQg1leAiu6ET"
API_SECRET = "GSq3rZN0d86zkBDx6s8JkI5ZkAIaHJ5897hlwluy4ibbLNLlcM"
BEARER_TOKEN = "AAAAAAAAAAAAAAAAAAAAAGRZZgEAAAAASCZ1Mw9xjXI6cyc7RMI9Lvurv%2FY%3DOtmBNmZk8Qgw1NaQ2wXtGfkICUU1Ng7mhbPDscuiqGiqdPEU3z"


In [ ]:
### making a test request to verify authentication

import tweepy

bearer_token = BEARER_TOKEN

client = tweepy.Client(bearer_token=bearer_token)

try:
    client.get_user(username="TwitterDev")  # Test request
    print("API Authentication Successful")
except Exception as e:
    print(f" Authentication Failed: {e}")


In [ ]:
### Authenticate with Twitter API

client = tweepy.Client(bearer_token=BEARER_TOKEN)

In [ ]:
### some widely recognized terms in mental health discussions

keywords = [
    "depression",
    "anxiety disorder",
    "bipolar disorder",
    "post-traumatic stress disorder",
    "obsessive-compulsive disorder",
    "schizophrenia",
    "eating disorder",
    "substance use disorder",
    "self-harm",
    "suicidal ideation"
]


In [ ]:
### Constructing the Search Query

query = " OR ".join(keywords) + " -is:retweet lang:en"

In [ ]:
### Retry logic for handling rate limits or temporary failures

max_retries = 5  # Number of retry attempts
retry_delay = 15  # Initial delay in seconds

for attempt in range(max_retries):
    try:
        # Fetch tweets (max 100 per request)
        tweets = client.search_recent_tweets(
            query=query,
            tweet_fields=["id", "text", "created_at", "public_metrics"],
            max_results=100
        )
        break  # If successful, exit loop

    except tweepy.TooManyRequests:  # Handle rate limits
        print(f"Rate limit exceeded. Retrying in {retry_delay} seconds...")
        time.sleep(retry_delay)
        retry_delay *= 2  # Exponential backoff

    except tweepy.TweepyException as e:  # Catch general Tweepy errors
        print(f"Error: {e}")
        time.sleep(retry_delay)


In [ ]:
# Store extracted data
data = []
for tweet in tweets.data:
    data.append({
        "Post ID": tweet.id,
        "Timestamp": tweet.created_at,
        "Content": tweet.text,
        "Likes": tweet.public_metrics["like_count"],
        "Replies": tweet.public_metrics["reply_count"],
        "Retweets": tweet.public_metrics["retweet_count"]
    })


### was getting requests issues, hence shifted to reddit API

In [12]:
### Reddit Data Extraction

import praw
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
import emoji
import os

from datetime import datetime

# Ensure stopwords are downloaded
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))



[nltk_data] Downloading package stopwords to /home/aakash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# Initialize Reddit API connection

user_agent= "user_agent"  #no space

reddit = praw.Reddit(
    client_id="-ggPSBpmj6tuwdfm-Av6vQ",
    client_secret="hP__xX396bg68yxyS9GzsCogUywMUQ",
    user_agent=user_agent
)


In [23]:
### some widely recognized terms in mental health discussions

keywords = [
    "depression",
    "anxiety disorder",
    "bipolar disorder",
    "post-traumatic stress disorder",
    "obsessive-compulsive disorder",
    "schizophrenia",
    "eating disorder",
    "substance use disorder",
    "self-harm",
    "suicidal ideation"
]


In [15]:
# List of subreddits to extract from
subreddit_name = [
    "mentalhealth", "depression", "addiction", "SuicideWatch",
    "traumatoolbox", "socialanxiety", "Anger", "offmychest", 
    "bodyacceptance", "mentalhealthmemes", 
    "nosurf", "mentalhealth"
]


In [16]:
import requests

def is_subreddit_accessible(subreddit_name):
    url = f"https://www.reddit.com/r/{subreddit_name}/about.json"
    headers = {"User-Agent": "your_user_agent"}
    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        return True  # ✅ Subreddit is accessible
    else:
        print(f"❌ Subreddit r/{subreddit_name} is private, banned, or doesn't exist.")
        return False  # ❌ Remove this subreddit from the list



In [17]:
# ✅ Filter only accessible subreddits


valid_subreddits = [sub for sub in subreddit_name if is_subreddit_accessible(sub)]
print(f"✅ Valid subreddits: {valid_subreddits}")

❌ Subreddit r/mentalhealthmemes is private, banned, or doesn't exist.
✅ Valid subreddits: ['mentalhealth', 'depression', 'addiction', 'SuicideWatch', 'traumatoolbox', 'socialanxiety', 'Anger', 'offmychest', 'bodyacceptance', 'nosurf', 'mentalhealth']


In [18]:
# Function to clean text

def clean_text(text):
    text = text.lower()
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)  # Remove special characters
    text = emoji.replace_emoji(text, '')  # Correct way to remove emojis
    text = ' '.join(word for word in text.split() if word not in stop_words)  # Remove stopwords
    return text


In [19]:
def get_reddit_posts(valid_subreddits, limit=100):
    all_posts = []
    
    for subreddit_name in valid_subreddits:
        print(f"🔹 Fetching posts from r/{subreddit_name}...")
        subreddit = reddit.subreddit(subreddit_name)
        
        for post in subreddit.hot(limit=limit):  # Fetch top 'hot' posts
            if any(keyword in post.title.lower() or keyword in post.selftext.lower() for keyword in keywords):
                all_posts.append({
                    "subreddit": subreddit_name,  # ✅ Store the subreddit name
                    "post_id": post.id,
                    "timestamp": datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),
                    "content": clean_text(post.title + ' ' + post.selftext),
                    "upvotes": post.score,
                    "comments": post.num_comments,
                    "shares": post.num_crossposts
                })
    
    return all_posts


In [24]:
# Fetch posts
posts_data = get_reddit_posts(valid_subreddits, limit=100)
print(f"✅ Collected {len(posts_data)} posts.")


🔹 Fetching posts from r/mentalhealth...


/tmp/ipykernel_22197/421323477.py:13: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  "timestamp": datetime.utcfromtimestamp(post.created_utc).strftime('%Y-%m-%d %H:%M:%S'),


🔹 Fetching posts from r/depression...
🔹 Fetching posts from r/addiction...
🔹 Fetching posts from r/SuicideWatch...
🔹 Fetching posts from r/traumatoolbox...
🔹 Fetching posts from r/socialanxiety...
🔹 Fetching posts from r/Anger...
🔹 Fetching posts from r/offmychest...
🔹 Fetching posts from r/bodyacceptance...
🔹 Fetching posts from r/nosurf...
🔹 Fetching posts from r/mentalhealth...
✅ Collected 108 posts.


In [26]:
# Convert the collected data to a DataFrame
df = pd.DataFrame(posts_data)

# Save the dataset to a CSV file
filename = "reddit_posts.csv"
df.to_csv(filename, index=False, encoding="utf-8")
print(f"✅ Data saved as {filename}")


✅ Data saved as reddit_posts.csv


In [27]:
df.head()

,subreddit,post_id,timestamp,content,upvotes,comments,shares
0,mentalhealth,1jmfpse,2025-03-29 05:50:57,girlfriend undergoing ect im 23m ive known gir...,41,47,0
1,mentalhealth,1jmheax,2025-03-29 07:55:47,think everyday tw read selfharm suicidal thoug...,5,1,0
2,mentalhealth,1jmku7i,2025-03-29 12:07:07,stop selfharm stopped cutting years ago cant s...,2,5,0
3,mentalhealth,1jmglfq,2025-03-29 06:54:51,see psych hey yall ive currently got issue par...,3,3,0
4,mentalhealth,1jmmpb3,2025-03-29 13:50:21,feel void emptiness inside seem go away 21f ep...,1,1,0


In [28]:
df.shape

(108, 7)

In [30]:
print(df.info())  

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   subreddit  108 non-null    object
 1   post_id    108 non-null    object
 2   timestamp  108 non-null    object
 3   content    108 non-null    object
 4   upvotes    108 non-null    int64 
 5   comments   108 non-null    int64 
 6   shares     108 non-null    int64 
dtypes: int64(3), object(4)
memory usage: 6.0+ KB
None


In [31]:
print(df.isnull().sum())  # Check missing values

subreddit    0
post_id      0
timestamp    0
content      0
upvotes      0
comments     0
shares       0
dtype: int64


In [33]:
df['content_length'] = df['content'].apply(len)
print(df['content_length'].describe())  # Get min, max, mean length of posts


count      108.000000
mean      1573.055556
std       2214.702933
min         49.000000
25%        553.000000
50%        859.000000
75%       1635.000000
max      14410.000000
Name: content_length, dtype: float64


In [34]:
print(df.sort_values(by='upvotes', ascending=False).head(5))  # Top 5 upvoted
print(df.sort_values(by='comments', ascending=False).head(5))  # Top 5 commented


     subreddit  post_id            timestamp  \
15  depression   doqwow  2019-10-29 14:52:02   
85  offmychest  1jlreto  2025-03-28 09:50:15   
38  depression  1jlk9ub  2025-03-28 02:02:08   
17  depression  1jm9yrj  2025-03-29 00:24:32   
20  depression  1jm35st  2025-03-28 19:19:08   

                                              content  upvotes  comments  \
15  mostbroken leastunderstood rules helpers may i...     2364       177   
85  spent 1600 savings porn 2 hours regret throwaw...     1814       299   
38  tonight im going kill name gian im peru unfort...      451       129   
17  dad said depression choice choosing depression...      132        89   
20  possible overcome depression im tired depresse...       97        51   

    shares  content_length  
15       2            3202  
85       0            1195  
38       0             939  
17       0              97  
20       0             142  
     subreddit  post_id            timestamp  \
85  offmychest  1jlreto  2025-03

In [ ]:
###  Cleaning the Text


In [51]:
import nltk

nltk.data.path.append('/home/aakash/nltk_data')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')


[nltk_data] Downloading package punkt to /home/aakash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/aakash/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/aakash/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [52]:
# 🔹 Load stopwords
stop_words = set(stopwords.words('english'))

# 🔹 Initialize lemmatizer
lemmatizer = WordNetLemmatizer()

# 🔹 Function to clean text
def clean_text(text):
    if pd.isna(text):  # Handle NaN values
        return ""
    
    text = text.lower()  # Convert to lowercase
    text = re.sub(r'[^a-z\s]', '', text)  # Remove special characters
    tokens = word_tokenize(text)  # Tokenize text
    tokens = [word for word in tokens if word not in stop_words]  # Remove stopwords
    tokens = [lemmatizer.lemmatize(word) for word in tokens]  # Apply lemmatization
    
    return ' '.join(tokens)




In [53]:
import nltk
nltk.data.path.append('/home/aakash/nltk_data')
nltk.download('punkt', download_dir='/home/aakash/nltk_data')

[nltk_data] Downloading package punkt to /home/aakash/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [54]:
df['content']

0      girlfriend undergoing ect im 23m ive known gir...
1      think everyday tw read selfharm suicidal thoug...
2      stop selfharm stopped cutting years ago cant s...
3      see psych hey yall ive currently got issue par...
4      feel void emptiness inside seem go away 21f ep...
                             ...                        
103    seeing hate affecting mental health noticed be...
104    clozapine escitalopram anxiety anyone share ex...
105    depression starting times really feel chance p...
106    mental health issues caused single lonely mala...
107    im point food thing makes happy life ive strug...
Name: content, Length: 108, dtype: object

In [55]:
# 🔹 Apply cleaning to dataset
df['cleaned_content'] = df['content'].apply(clean_text)

# 🔹 Save cleaned dataset
df.to_csv("cleaned_reddit_posts.csv", index=False)
print("✅ Cleaned dataset saved as cleaned_reddit_posts.csv")


✅ Cleaned dataset saved as cleaned_reddit_posts.csv


In [57]:
# 🔹 Drop the old content column
df = df.drop(columns=['content'])

# 🔹 Save the cleaned dataset
df.to_csv("cleaned_reddit_posts.csv", index=False)
print("✅ Cleaned dataset saved with only cleaned content.")


✅ Cleaned dataset saved with only cleaned content.


In [58]:
df.head()

,subreddit,post_id,timestamp,upvotes,comments,shares,content_length,cleaned_content
0,mentalhealth,1jmfpse,2025-03-29 05:50:57,41,47,0,747,girlfriend undergoing ect im ive known girlfri...
1,mentalhealth,1jmheax,2025-03-29 07:55:47,5,1,0,978,think everyday tw read selfharm suicidal thoug...
2,mentalhealth,1jmku7i,2025-03-29 12:07:07,2,5,0,121,stop selfharm stopped cutting year ago cant st...
3,mentalhealth,1jmglfq,2025-03-29 06:54:51,3,3,0,550,see psych hey yall ive currently got issue par...
4,mentalhealth,1jmmpb3,2025-03-29 13:50:21,1,1,0,864,feel void emptiness inside seem go away f epis...


In [59]:
df['word_count'] = df['cleaned_content'].apply(lambda x: len(x.split()))
print(df['word_count'].describe())


count     108.000000
mean      225.805556
std       303.378837
min         9.000000
25%        88.000000
50%       126.000000
75%       241.000000
max      1940.000000
Name: word_count, dtype: float64


# Task 2

In [ ]:
### 🔹 Step 1: Sentiment Analysis 


In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import pandas as pd 


In [ ]:
# 🔹 Load pre-trained BERT sentiment model (DistilBERT-SST-2)
MODEL_NAME = "distilbert-base-uncased-finetuned-sst-2-english"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForSequenceClassification.from_pretrained(MODEL_NAME)

In [ ]:

# 🔹 Function to classify sentiment
def classify_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True, max_length=512)
    with torch.no_grad():
        outputs = model(**inputs)
    probs = torch.nn.functional.softmax(outputs.logits, dim=-1)
    sentiment = "Positive" if torch.argmax(probs) == 1 else "Negative"
    return sentiment


In [ ]:


# 🔹 Load your cleaned Reddit dataset
df = pd.read_csv("cleaned_reddit_posts.csv")

In [ ]:
## Step 2: Crisis Risk Classification (TF-IDF & Rule-Based Detection)


In [ ]:
high_risk_terms = [
    "i don’t want to be here", "i want to end it", "life is meaningless", 
    "no reason to live", "i can’t go on", "goodbye everyone", "ending it all"
]

moderate_risk_terms = [
    "i feel lost", "struggling", "overwhelmed", "i need help", "nobody understands", 
    "depressed", "can’t cope", "feeling empty"
]

# Function to classify crisis risk level
def classify_risk(text):
    text_lower = text.lower()
    if any(term in text_lower for term in high_risk_terms):
        return "High-Risk"
    elif any(term in text_lower for term in moderate_risk_terms):
        return "Moderate Concern"
    else:
        return "Low Concern"

# Apply risk classification
df["Risk_Level"] = df["Cleaned_Content"].apply(classify_risk)

# Save the updated dataset
df.to_csv("classified_twitter_data.csv", index=False)
print("✅ Crisis risk classification done!")


In [ ]:
df.head()

In [ ]:
### 🔹 Step 3: Visualizing Sentiment & Risk Distribution


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set style
sns.set_style("darkgrid")

# Plot sentiment distribution
plt.figure(figsize=(8, 4))
sns.countplot(x=df["Sentiment"], palette="coolwarm")
plt.title("Sentiment Analysis Distribution")
plt.show()

# Plot risk level distribution
plt.figure(figsize=(8, 4))
sns.countplot(x=df["Risk_Level"], palette="Reds")
plt.title("Crisis Risk Classification Distribution")
plt.show()


In [ ]:
####### Task 3

In [ ]:
# Step 1: Extracting Location Data
##

In [ ]:
# Check available location data
print(df["Location"].value_counts().head(10))  # View top locations


In [ ]:
###📌 Extract Locations from Post Text


In [ ]:
import spacy

# Load English NLP model
nlp = spacy.load("en_core_web_sm")

# Function to extract locations
def extract_location(text):
    doc = nlp(text)
    for ent in doc.ents:
        if ent.label_ == "GPE":  # GPE = Geo-Political Entity (cities, countries, etc.)
            return ent.text
    return None  # No location found

# Apply location extraction
df["Extracted_Location"] = df["Cleaned_Content"].apply(extract_location)

# View extracted locations
print(df["Extracted_Location"].value_counts().head(10))


In [ ]:
## Step 2: Geocoding (Convert Locations to Lat/Lon)

In [ ]:
from geopy.geocoders import Nominatim
import time

geolocator = Nominatim(user_agent="geo_locator")

# Function to get latitude & longitude
def get_coordinates(location):
    if pd.isna(location):
        return None, None
    try:
        time.sleep(1)  # To avoid API rate limits
        loc = geolocator.geocode(location)
        if loc:
            return loc.latitude, loc.longitude
    except:
        return None, None
    return None, None

# Apply geocoding
df[["Latitude", "Longitude"]] = df["Extracted_Location"].apply(lambda x: pd.Series(get_coordinates(x)))

# Remove rows where coordinates are missing
df = df.dropna(subset=["Latitude", "Longitude"])

# Save the geocoded dataset
df.to_csv("geocoded_twitter_data.csv", index=False)
print("✅ Geolocation completed!")


In [ ]:
### 🔹 Step 3: Creating a Crisis Heatmap


In [ ]:
## generate heatmap

In [ ]:
import folium
from folium.plugins import HeatMap

# Initialize map centered at an approximate location
m = folium.Map(location=[20, 0], zoom_start=2)  # World view

# Add heatmap layer
heat_data = df[["Latitude", "Longitude"]].values.tolist()
HeatMap(heat_data).add_to(m)

# Save and display map
m.save("crisis_heatmap.html")
print("✅ Heatmap generated! Open 'crisis_heatmap.html' to view.")


In [ ]:
## 🔹 Step 4: Finding Top 5 Crisis Locations



In [ ]:
# Count top locations
top_locations = df["Extracted_Location"].value_counts().head(5)
print(top_locations)
